In [14]:
import numpy as np
import math 
from scipy.stats import norm
from scipy.stats import truncnorm

In [24]:
def autoselect_scales_mix_norm(betahat, sebetahat, max_class=None, mult=2):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = 2*np.sqrt(np.max(betahat**2 - sebetahat**2))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
        
    
    return out
     
def autoselect_scales_mix_exp(betahat, sebetahat, max_class=None , mult=1.5,tt=1.5):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = tt*np.sqrt(np.max(betahat**2  ))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
                if(out[2] <1e-2 ):
                 out[2: ] <- out[2: ] +1e-2
         
    
    return out

    
def wpost_exp ( x, s, w, scale):
    
    if  w[0]==1:
     out =  np.concatenate(([1]  ,np.full( scale.shape[0],[0])))
     return out
    else:
     a=1/scale[1:]
     w = assignment
     a = 1 / scale[1:]  # Note: slicing in Python is zero-based, so [1:] starts from the second element
     lf = norm.logpdf(x, loc=0, scale=s)
     lg = np.log(a) + s**2 * a**2 / 2 - a * x + norm.logcdf(x / s - s * a)
     log_prob = np.concatenate(([lf]  ,lg ))
     bmax=np.max(log_prob)
     log_prob = log_prob - bmax
 
     log_prob = log_prob - bmax
     wpost = w* np.exp( log_prob) / (sum(w *np.exp(log_prob)))
     return wpost    
 
 
def expectation_truncated(a, b, mean, sd):
    # Standardize the bounds
    alpha = (a - mean) / sd
    beta = (b - mean) / sd
    
    # Calculate the PDF and CDF of the standard normal distribution
    phi_alpha = truncnorm.pdf(alpha, alpha, beta)
    phi_beta = truncnorm.pdf(beta, alpha, beta)
    Phi_alpha = truncnorm.cdf(alpha, alpha, beta)
    Phi_beta = truncnorm.cdf(beta, alpha, beta)
    
    # Calculate the expected value
    expected_value = mean + (phi_alpha - phi_beta) / (Phi_beta - Phi_alpha) * sd
    
    return expected_value

In [27]:
# Example usage

expected_value =expectation_truncated(0,2,3,1)
print(f"Expected value of the truncated normal distribution: {expected_value}")

Expected value of the truncated normal distribution: 3.004437839042126


should equal to 1.4899504867560165

In [7]:
betahat=  np.array([1,2,3,4,5])
sebetahat=np.array([1,0.4,5,1,1])
scale = autoselect_scales_mix_exp ( np.array([1,2,3,4,5]),  np.array([1,0.4,5,1,1]))

non_informativ = np.full( scale.shape[0], 1/ scale.shape[0])
n=betahat.shape[0]
log_pi =  np.log( np.tile(non_informativ, (n, 1)))

In [8]:
assignment = np.exp(log_pi)[0]


assignment = assignment /   sum(assignment)
print(assignment)
x=betahat[1]
s=sebetahat[1]

[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]


writing wpost_def

In [10]:
w=assignment
print(w)
wpost_exp ( x, s, w, scale)
temp_array =   np.zeros ( (betahat.shape[0], scale.shape[0]))
i=1
temp_array[i,] = wpost_exp ( x=betahat[i], s=sebetahat[i], w=w, scale=scale) 

[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]


should  be array([3.53987758e-06, 6.61493885e-06, 1.06391083e-05, 2.86976960e-05,
       1.69165759e-04, 1.40776600e-03, 8.91255655e-03, 3.45101678e-02,
       8.34205613e-02, 1.38160703e-01, 1.72844260e-01, 1.77093219e-01,
       1.57939387e-01, 1.28091688e-01, 9.74010338e-02])

In [13]:
print(scale.shape[0] )
temp_array =   np.zeros ( (betahat.shape[0], scale.shape[0]))
for i in range(betahat.shape[0]):
    temp_array[i,] = wpost_exp ( x=betahat[i], s=sebetahat[i], w=np.exp(log_pi)[i,], scale=scale) 
print(temp_array)

temp_array[0,]== wpost_exp ( x=betahat[0], s=sebetahat[0], w=w, scale=scale) 


15
[[6.52918851e-02 6.78002962e-02 6.90396965e-02 7.08629747e-02
  7.34774073e-02 7.70129806e-02 8.12078860e-02 8.48923837e-02
  8.58678052e-02 8.20342129e-02 7.30701767e-02 6.07291363e-02
  4.75513198e-02 3.55270606e-02 2.56347785e-02]
 [3.53987758e-06 6.61493885e-06 1.06391083e-05 2.86976960e-05
  1.69165759e-04 1.40776600e-03 8.91255655e-03 3.45101678e-02
  8.34205613e-02 1.38160703e-01 1.72844260e-01 1.77093219e-01
  1.57939387e-01 1.28091688e-01 9.74010338e-02]
 [6.56166419e-02 6.59175381e-02 6.60660268e-02 6.62862131e-02
  6.66105355e-02 6.70828769e-02 6.77572351e-02 6.86850138e-02
  6.98706687e-02 7.11576607e-02 7.20227505e-02 7.14207842e-02
  6.80838604e-02 6.13953338e-02 5.20268604e-02]
 [2.97209370e-04 3.50647498e-04 3.84449298e-04 4.47398412e-04
  5.81502557e-04 9.36473754e-04 2.17223266e-03 7.15579060e-03
  2.44160538e-02 6.53144569e-02 1.26462752e-01 1.82759260e-01
  2.09880744e-01 2.03590339e-01 1.75250690e-01]
 [4.80878262e-06 5.94302299e-06 6.71979846e-06 8.30282145e-06

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [22]:
post_mean = np.zeros(betahat.shape[0])
print(post_mean)

for i in range(post_mean.shape[0]):
    post_mean[i]= expectation_truncated(0, np.inf, betahat[i]- sebetahat** 2 )


[0. 0. 0. 0. 0.]


In [33]:
scale

array([0.03853673, 0.0578051 , 0.08670765, 0.13006147, 0.19509221,
       0.29263832, 0.43895748, 0.65843621, 0.98765432, 1.48148148,
       2.22222222, 3.33333333, 5.        , 7.5       ])

In [36]:
i=1
print(expectation_truncated(0, np.inf, betahat[i]- sebetahat[i]** 2 *(1/scale[1:]) , sebetahat[i]).sum)

<built-in method sum of numpy.ndarray object at 0x00000259405BA430>
